# BLa BLA BLA

... ... 

In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from math import log, sqrt
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pk
import numpy as np
import glob
import os
from collections import Counter
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
#%%

In [11]:
df = pd.read_csv('childhood.csv', delimiter=';', encoding='iso-8859-1')

In [12]:
df.to_csv('childhood.csv', encoding='utf-8')

In [10]:
pd.read_csv('hpv.csv')

,Unnamed: 0,Zoekterm,Nr,Datum,Type,URL,webcitation url,Misinfo Nee(1) Ja(0),Platte tekst (zonder opmaak),Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,0,NaN,Nummer elke blog/website,NaN,blog (interactief [je kunt reageren; vaak een ...,NaN,Gebruik website om de url te krijgen: webcitat...,Is de conclusie in lijn met de richtlijnen ? Z...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,HPV vaccinaties veilig,1,16-4-2019,Website,https://rijksvaccinatieprogramma.nl/nieuws/ond...,http://www.webcitation.org/query?url=https%3A%...,1,Onderzoek van het RIVM Rijksinstituut voor Vol...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,HPV vaccinaties veilig,2,16-4-2019,Website,https://rijksvaccinatieprogramma.nl/vaccinatie...,http://www.webcitation.org/query?url=https%3A%...,1,HPV is de afkorting van humaan papillomavirus....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,HPV vaccinaties veilig,3,16-4-2019,Website,https://www.rivm.nl/veelgestelde-vragen-over-h...,http://www.webcitation.org/query?url=https%3A%...,1,Hieronder vind je een selectie van vragen en a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,HPV vaccinaties veilig,4,16-4-2019,Website,https://stichtingvaccinvrij.nl/hpv-vaccin/,http://www.webcitation.org/query?url=https%3A%...,0,Wat is de noodzaak van het HPV vaccin? Wat is ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,HPV vaccinaties veilig,5,16-4-2019,Website,https://ggd.groningen.nl/hpv-vaccin-is-veilig/,http://www.webcitation.org/query?url=https%3A%...,1,HPV-vaccin is veilig\nGeplaatst op: 25 septemb...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,HPV vaccinaties veilig,6,16-4-2019,Website,https://www.kwf.nl/voorkomen/hpv-vaccinatie/Pa...,http://www.webcitation.org/query?url=https%3A%...,1,Alles wat je moet weten over de HPV-vaccinatie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,HPV vaccinaties veilig,7,16-4-2019,Website,https://www.kwf.nl/voorkomen/hpv-vaccinatie/Pa...,http://www.webcitation.org/query?url=https%3A%...,1,HPV-vaccinatie beschermt tegen baarmoederhalsk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,HPV vaccinaties veilig,9,16-4-2019,Website,https://www.soaaids.nl/nl/soas/voorkom-soas/va...,http://www.webcitation.org/query?url=https%3A%...,1,Vaccineren HPV\nMeisjes vanaf 12 jaar die nog ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
